In [3]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from functions.input_functions import display_widgets

df = pd.read_csv("cleaned_zomato.csv")
df["cuisines"] = df["cuisines"].apply(eval)

In [4]:
display_widgets(df)

SelectMultiple(description='Cuisines', options=('North Indian', 'Chinese', 'South Indian', 'Fast Food', 'Birya…

FloatSlider(value=4.0, description='Minimum rate: ', max=5.0)

SelectionRangeSlider(description='Price range: ', index=(0, 3), options=('$', '$$', '$$$', '$$$$'), value=('$'…

Button(description='OK', style=ButtonStyle())

Your preferences:
 -- cuisine: ('Chinese',) 
 -- minimum rating: 4.0 
 -- price: ('$', '$$')
